# Trích xuất khuôn mặt trực tiếp từ camera

# I. Thư viện

In [1]:
from all_lib import *

# II. Chuẩn bị

Gọi GPU

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
data_path = '../data/annotations'

Tải dữ liệu

In [4]:
embeddings= torch.load(data_path + '/facelist.pth')
names = np.load(data_path + '/username.npy')

Kích thước đầu vào lấy từ webcam

In [5]:
frame_size =  (640, 480)

# III Chuẩn hóa

Model MTCNN để tách khuôn mặt

In [6]:
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)

Các hàm chuẩn hóa

In [7]:
def standardization(img_tensor_tmp):
    normalize = (img_tensor_tmp -127.5)/128
    return normalize

In [8]:
def trans(img):
    transform = transforms.Compose([
        transforms.ToTensor(),
        fixed_image_standardization
    ])
    return transform(img)

# IV. Main

Khởi tạo model InceptionResnetV1

In [9]:
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

Lấy camera

In [10]:
video_cap = cv2.VideoCapture(0)
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

True

### Funtion

extract_face: Trích xuất khuôn mặt từ các bounding box
Thông số:
- margin: Tương đương với margin khi capture_face ( để là 20)


In [11]:
def extract_face(box, img, margin=20):
    face_size = 160
    img_size = frame_size
    margin = [
        margin * (box[2] - box[0]) / (face_size - margin),
        margin * (box[3] - box[1]) / (face_size - margin),
    ] #tạo margin bao quanh box cũ
    box = [
        int(max(box[0] - margin[0] / 2, 0)),
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, img_size[0])),
        int(min(box[3] + margin[1] / 2, img_size[1])),
    ]
    img = img[box[1]:box[3], box[0]:box[2]]
    face = cv2.resize(img,(face_size, face_size), interpolation=cv2.INTER_AREA)
    face = Image.fromarray(face)
    return face

Kết xuất embedding cho từng ảnh mặt - thứ đã được extract từ hàm extract_face() ở trên

In [12]:
def inference(model, face, local_embeds, threshold = 3):
    #local: [n,512] voi n la so nguoi trong faceslist
    embeds = []
    # print(trans(face).unsqueeze(0).shape)
    embeds.append(model(trans(face).to(device).unsqueeze(0)))
    detect_embeds = torch.cat(embeds) #[1,512]
    # print(detect_embeds.shape)
    #[1,512,1]                                      [1,512,n]
    norm_diff = detect_embeds.unsqueeze(-1) - torch.transpose(local_embeds, 0, 1).unsqueeze(0)
    # print(norm_diff)
    norm_score = torch.sum(torch.pow(norm_diff, 2), dim=1) #(1,n), moi cot la tong khoang cach euclide so vs embed moi

    min_dist, embed_idx = torch.min(norm_score, dim = 1)
    print(min_dist*power, names[embed_idx])
    # print(min_dist.shape)
    if min_dist*power > threshold:
        return -1, -1
    else:
        return embed_idx, min_dist.double()

main

In [13]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

In [14]:
x = [1200, 980, 800, 680, 580, 520, 448, 412, 372, 348, 320, 300, 280, 268, 248, 236, 228]
y = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
coff = np.polyfit(x, y, 2)  # y = Ax^2 + Bx + C
A, B, C = coff

In [15]:
prev_frame_time = 0
new_frame_time = 0
power = pow(10, 6)

In [ ]:
while video_cap.isOpened():
    isSuccess, frame = video_cap.read()
    imgRGB= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    cvzone.putTextRect(frame,'Trang thai', (2,40), colorR=(14,201,255))
    #text_status = 'Dang Tim'
    if results.pose_landmarks:
        mpDraw.draw_landmarks(frame,results.pose_landmarks,mpPose.POSE_CONNECTIONS )
        for index ,dot in enumerate(results.pose_landmarks.landmark):
            if(index == 11):
                hight ,weight,c=frame.shape
                center_x1, center_y1 =int(dot.x*weight),int(dot.y*hight)
                cv2.circle(frame,(center_x1,center_y1),5,(0,0,255),cv2.FILLED)

            if(index == 12):
                hight ,weight,c=frame.shape
                center_x2, center_y2 =int(dot.x*weight),int(dot.y*hight)
                cv2.circle(frame,(center_x2,center_y2),5,(0,0,255),cv2.FILLED)
        distance = int(math.sqrt((center_y2 - center_y1) ** 2 + (center_x2 - center_x1) ** 2))
        distanceCM = A * distance ** 2 + B * distance + C
        cvzone.putTextRect(frame, f'Khoang cach:{int(distanceCM)} ', (1,95), colorR=(14,201,255))
        # nhan dien khach hang
        boxes, _ = mtcnn.detect(frame)
        if boxes is not None:
            for box in boxes:
                bbox = list(map(int,box.tolist()))
                face = extract_face(bbox, frame)
                idx, score = inference(model, face, embeddings)
                if idx != -1:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (255,201,14), 4)
                    score = torch.Tensor.cpu(score[0]).detach().numpy()*power
                    cvzone.putTextRect(frame, names[idx] + '_{:.2f}'.format(score), (bbox[0],bbox[1]), colorR=(255,201,14))
                    cvzone.putTextRect(frame, 'Da tim thay' , (290,40), colorR=(0,255,0))
                    cvzone.putTextRect(frame, 'Dung' , (1,480), colorR=(14,201,255))
                else:
                    frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (255,201,14), 4)
                    cvzone.putTextRect(frame, 'Unknown', (bbox[0],bbox[1]), colorR=(255,201,14))
                    cvzone.putTextRect(frame, 'Nham nguoi' , (290,40), colorR=(0,0,255))
                    cvzone.putTextRect(frame, 'Lui' , (1,480), colorR=(14,201,255))
        else:
            cvzone.putTextRect(frame, 'Nhan dang' , (290,40), colorR=(0,0,255))
            cvzone.putTextRect(frame, 'Tien' , (1,480), colorR=(14,201,255))
    else:
        cvzone.putTextRect(frame, 'Dang tim' , (290,40), colorR=(0,0,255))



    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break

tensor([0.1197], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3757], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2649], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2401], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3414], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3698], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.4206], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.2619], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3076], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3259], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3665], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3478], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3408], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.4149], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3059], device='cuda:0', grad_fn=<MulBackward0>) Cong
tensor([0.3415], device='cuda:0', grad_fn=<MulBackward0

In [ ]:
video_cap.release()
cv2.destroyAllWindows()